In [2]:
!nvidia-smi

Mon Nov 23 20:32:26 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!pip install transformers

In [1]:
import pandas as pd
import numpy as np
import torch

from transformers import (XLMRobertaTokenizer, 
                          XLMRobertaForSequenceClassification, 
                          Trainer, TrainingArguments)
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tqdm.notebook import tqdm

In [2]:
root_path = '/content/drive/MyDrive/wow/'

In [3]:
train, test = [pd.read_csv(root_path + f'{fname}.csv', sep='\t') for fname in ['train', 'test']]

In [4]:
train

,text,class
0,В Синьцзян-Уйгурском автономном районе Китая С...,true
1,Режиссер Дэвид Линч один из создателей телесер...,true
2,Министр регионального развития России Виктор Б...,true
3,На несогласованной акции в поддержку украинско...,true
4,Госдума РФ одобрила в первом чтении законопрое...,true
...,...,...
23878,Пожар на плавучем дебаркадере в центре Москвы ...,true
23879,В Омске возбудили уголовное дело в отношении 6...,true
23880,\n var PROJECTID 80674\n function w d c...,true
23881,Бывший главный тренер московского футбольного ...,true


In [8]:
train.shape

(23883, 2)

In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23883 entries, 0 to 23882
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    23878 non-null  object
 1   class   23883 non-null  object
dtypes: object(2)
memory usage: 373.3+ KB


In [10]:
train['class'].value_counts()

true    19919
fake     3964
Name: class, dtype: int64

In [5]:
train.dropna(inplace=True)

In [12]:
test

,text,class
0,Британское министерство информации приняло дек...,NaN
1,"Альбом ""Chief"" кантри-исполнителя Эрика Черча ...",NaN
2,Новый вид микрокредитных организаций появился ...,NaN
3,Один из крупнейших торговых комплексов Москвы ...,NaN
4,Министерство обороны Белоруссии приостановила ...,NaN
...,...,...
6019,Представители интернет-сообщества попросили Го...,NaN
6020,Московские кришнаиты как и было запланировано ...,NaN
6021,Компания Fox Broadcasting договорились с опера...,NaN
6022,Гражданка Великобритании Джорджина Чайлдс задо...,NaN


In [6]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6024 entries, 0 to 6023
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   text    6024 non-null   object 
 1   class   0 non-null      float64
dtypes: float64(1), object(1)
memory usage: 94.2+ KB


In [7]:
test.drop(columns=['class'], inplace=True)
test

,text
0,Британское министерство информации приняло дек...
1,"Альбом ""Chief"" кантри-исполнителя Эрика Черча ..."
2,Новый вид микрокредитных организаций появился ...
3,Один из крупнейших торговых комплексов Москвы ...
4,Министерство обороны Белоруссии приостановила ...
...,...
6019,Представители интернет-сообщества попросили Го...
6020,Московские кришнаиты как и было запланировано ...
6021,Компания Fox Broadcasting договорились с опера...
6022,Гражданка Великобритании Джорджина Чайлдс задо...


In [8]:
class FNDataset(torch.utils.data.Dataset):
  def __init__(self, encodings, labels):
    self.encodings = encodings
    self.labels = labels

  def __getitem__(self, idx):
    item = dict((key, torch.tensor(val[idx])) for key, val in self.encodings.items())
    item['labels'] = torch.tensor(self.labels[idx])

    return item

  def __len__(self):
    return len(self.labels)

In [9]:
x_train, x_test, y_train, y_test = train_test_split(train.text.astype(str).values, 
                                                    train['class'].apply(lambda x: int(x == 'true')).values,
                                                    test_size=0.05, random_state=42)

In [10]:
tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-large')

In [23]:
train_encodings = tokenizer(x_train.tolist(), truncation=True, padding=True)
test_encodings = tokenizer(x_test.tolist(), truncation=True, padding=True)
val_encodings = tokenizer(test.text.astype(str).values.tolist(), truncation=True, padding=True)

In [34]:
train_ds = FNDataset(train_encodings, y_train)
test_ds = FNDataset(test_encodings, [-1] * len(x_test))
val_ds = FNDataset(val_encodings, [-1] * len(test.text))

In [20]:
training_args = TrainingArguments(
    output_dir = root_path + 'result',
    num_train_epochs = 1,
    per_device_train_batch_size = 2,
    max_grad_norm = 0.25,
    gradient_accumulation_steps = 4,
    per_device_eval_batch_size = 64,
    warmup_steps = 1000,
    weight_decay = 0.01,
    logging_dir = root_path + 'logs',
    learning_rate = 3e-05,
    logging_steps = 25,
    save_steps = 5000,
    fp16 = True
)

In [21]:
model = XLMRobertaForSequenceClassification.from_pretrained('xlm-roberta-large')

Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.den

In [22]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_ds,
    eval_dataset = test_ds
)

trainer.train()

Step,Training Loss
25,0.799112
50,0.571265
75,0.469835
100,0.412116
125,0.337836
150,0.186908
175,0.122020
200,0.068196
225,0.056667
250,0.002781


TrainOutput(global_step=2835, training_loss=0.04150111553319968)

In [23]:
model.save_pretrained(root_path + 'xlm-roberta-large-1st')

In [13]:
model = XLMRobertaForSequenceClassification.from_pretrained(root_path + 'xlm-roberta-large-1st').cuda()

In [48]:
test_iter = torch.utils.data.DataLoader(test_ds, batch_size=32)
fin_iter = torch.utils.data.DataLoader(val_ds, batch_size=32)

In [47]:
def get_preds(iter, model):
  preds = []
  for batch in tqdm(iter):
    with torch.no_grad():
      preds.append(model(
          batch['input_ids'].cuda(),
          batch['attention_mask'].cuda()
      ))

  return preds

In [51]:
test_preds = get_preds(test_iter, model)
fin_preds = get_preds(fin_iter, model)

In [62]:
def bin_preds(preds):
  bin = []
  for batch in preds:
    bin += (torch.softmax(batch[0], dim=1) >= 0.5).cpu().numpy().astype(int).argmax(axis=1).tolist()
  
  return bin

In [63]:
test_bin_preds = bin_preds(test_preds)
fin_bin_preds = bin_preds(fin_preds)

In [64]:
f1_score(y_test, test_bin_preds)

0.998003992015968

In [41]:
test['class'] = pd.Series(fin_preds).map({1: 'true', 0: 'fake'})
test

,text,class
0,Британское министерство информации приняло дек...,fake
1,"Альбом ""Chief"" кантри-исполнителя Эрика Черча ...",true
2,Новый вид микрокредитных организаций появился ...,fake
3,Один из крупнейших торговых комплексов Москвы ...,fake
4,Министерство обороны Белоруссии приостановила ...,true
...,...,...
6019,Представители интернет-сообщества попросили Го...,true
6020,Московские кришнаиты как и было запланировано ...,true
6021,Компания Fox Broadcasting договорились с опера...,true
6022,Гражданка Великобритании Джорджина Чайлдс задо...,true


In [43]:
test.to_csv(root_path + 'test.csv', index=False, sep='\t')

In [45]:
from zipfile import ZipFile
with ZipFile(root_path + 'sub.zip', 'w') as s:
  s.write(root_path + 'test.csv', 'test.csv')